In [1]:
import pandas as pd
import numpy as np
import random
pd.set_option('display.width', None)

In [2]:
with open('../datasets/fraudTrain.csv') as fd:
    numcols = len(fd.readline().split(','))
df = pd.read_csv('../datasets/fraudTrain.csv', usecols=range(1,numcols))

In [3]:
display(df)

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,2020-06-21 12:12:08,30263540414123,fraud_Reichel Inc,entertainment,15.56,Erik,Patterson,M,162 Jessica Row Apt. 072,Hatch,...,37.7175,-112.4777,258,Geoscientist,1961-11-24,440b587732da4dc1a6395aba5fb41669,1371816728,36.841266,-111.690765,0
1296671,2020-06-21 12:12:19,6011149206456997,fraud_Abernathy and Sons,food_dining,51.70,Jeffrey,White,M,8617 Holmes Terrace Suite 651,Tuscarora,...,39.2667,-77.5101,100,"Production assistant, television",1979-12-11,278000d2e0d2277d1de2f890067dcc0a,1371816739,38.906881,-78.246528,0
1296672,2020-06-21 12:12:32,3514865930894695,fraud_Stiedemann Ltd,food_dining,105.93,Christopher,Castaneda,M,1632 Cohen Drive Suite 639,High Rolls Mountain Park,...,32.9396,-105.8189,899,Naval architect,1967-08-30,483f52fe67fabef353d552c1e662974c,1371816752,33.619513,-105.130529,0
1296673,2020-06-21 12:13:36,2720012583106919,"fraud_Reinger, Weissnat and Strosin",food_dining,74.90,Joseph,Murray,M,42933 Ryan Underpass,Manderson,...,43.3526,-102.5411,1126,Volunteer coordinator,1980-08-18,d667cdcbadaaed3da3f4020e83591c83,1371816816,42.788940,-103.241160,0


In [4]:
cc_info_table  = df.groupby(['cc_num','first', 'last', 'dob', 'street', 'city', 'state', 'zip'])[['trans_date_trans_time','amt']].apply(lambda x: x.values.tolist())
cc_info_table = cc_info_table.reset_index(name='transactions')
#display(cc_info_table)

In [5]:
df.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')

In [6]:
def extract_transaction_info(rec):
    date = str(rec[1]['trans_date_trans_time']).split(' ')[0]
    cc_num = str(rec[1]['cc_num'])
    amount = str(rec[1]['amt'])
    first_name = str(rec[1]['first'])
    last_name = str(rec[1]['last'])
    merchant = str(rec[1]['merchant'])
    if merchant.startswith('fraud_'):
        merchant = merchant[6:]
    category = str(rec[1]['category'])
    dob = str(rec[1]['dob'])
    trans_num = str(rec[1]['trans_num'])
    street = str(rec[1]['street'])
    city = str(rec[1]['city'])
    state = str(rec[1]['state'])
    zipcode = str(rec[1]['zip'])
    return (date, cc_num, amount, first_name, last_name, merchant, category, dob, trans_num, street, city, state, zipcode)
    

In [7]:
def create_type_1_dataset(df, instructions, inputs, outputs, texts):
    for rec in df.iterrows():
        date, cc_num, amount, first_name, last_name, merchant, category, dob, trans_num, street, city, state, zipcode = extract_transaction_info(rec)
        instruction = f"### INSTRUCTION:\nProvide the details of the transaction made using the Card Number, Transaction Date, and the Date of Birth of the user given in the INPUT. The Response must include Merchant Name, Category, Transaction Id, and the Amount spent. If you do not have some of the information then output NA for those fields. If you have additional information output that as Others."
        input       = f"### INPUT:\nCard Number: {cc_num}\nTransaction Date: {date}\nDate of Birth: {dob}"
        output      = f"### RESPONSE:\nMerchant: {merchant}\nCategory: {category}\nTransaction Id: {trans_num}\nAmount: ${amount}\nOthers: \n</s>"
        text        = f"The following is an INSTRUCTION for a task.The paired INPUT below provides additional context for the task. Write a response to complete the task.\n\n{instruction}\n\n{input}\n\n{output}"
        instructions.append(instruction)
        inputs.append(input)
        outputs.append(output)
        texts.append(text)


In [8]:
def create_type_2_dataset(df, instructions, inputs, outputs, texts):
    for rec in df.iterrows():
        date, cc_num, amount, first_name, last_name, merchant, category, dob, trans_num, street, city, state, zipcode = extract_transaction_info(rec)
        last4digits = cc_num[-4:]
        instruction = f"### INSTRUCTION:\nProvide details of the credit card transaction made by a user. The First Name of the user, Transaction Date, and the Merchant Name are given in the INPUT context. Please provide the Last Name of the user, the Last 4 digits of the Credit Card number used, the Street Address and the Zipcode of the user. If you do not have some of the information then output NA for those fields. If you have additional information output that as Others."
        input       = f"### INPUT:\nFirst Name: {first_name}\nTransaction Date: {date}\nMerchant Name: {merchant}"
        output      = f"### RESPONSE:\nLast Name: {last_name}\nLast 4 digits of the Credit Card: {last4digits}\nStreet Address: {street}\nZipcode: {zipcode}\nOthers: \n</s>"
        text        = f"The following is an INSTRUCTION for a task. The paired INPUT below provides additional context for the task. Write a response to complete the task.\n\n{instruction}\n\n{input}\n\n{output}"
        instructions.append(instruction)
        inputs.append(input)
        outputs.append(output)
        texts.append(text)

In [9]:
def create_type_3_dataset(instructions, inputs, outputs, texts):
    #cc_info_table  = df.groupby(['cc_num','first', 'last', 'dob', 'street', 'city', 'state', 'zip'])[['trans_date_trans_time','amt']].apply(lambda x: x.values.tolist())
    #cc_info_table = cc_info_table.reset_index(name='transactions')
    for r in cc_info_table.iterrows():
        cc_num     = r[1]['cc_num']
        first_name = r[1]['first']
        last_name  = r[1]['last']
        dob        = r[1]['dob']
        street     = r[1]['street']
        city       = r[1]['city']
        state      = r[1]['state']
        zipcode    = r[1]['zip']
        
        output     = f"### RESPONSE:\nCredit Card: {cc_num}\n"
        num        = min(len(r[1]['transactions']), 10)

        for indx, trans in enumerate(r[1]['transactions']):
            date = trans[0].split(' ')[0]
            amount = trans[1]
            output += f"Transaction Date: {date}, Amount: ${amount}\n"
            if indx >= num -1:
                break
        output += "</s> "
            
        instruction = f"### INSTRUCTION:\nFind all Credit Card transactions made by the given user. The First Name and the Last Name of the user, Date of Birth, Street Address, City, State, Zipcode are given in the INPUT context. Please provide the Credit Card number, and all the known Transaction Dates, and the Amount charged for each Transaction. If you do not have some of the information then output NA for those fields."
        input       = f"### INPUT:\nFirst Name: {first_name}\nLast Name: {last_name}\nDate of Birth: {dob}\nStreet Address: {street}\nCity: {city}\nState: {state}\nZipcode: {zipcode}\n"
        text        = f"The following is an INSTRUCTION for a task. The paired INPUT below provides additional context for the task. Write a response to complete the task.\n\n{instruction}\n\n{input}\n\n{output}"
        instructions.append(instruction)
        inputs.append(input)
        outputs.append(output)
        texts.append(text)

In [10]:
inputs       = []
outputs      = []
instructions = []
texts        = []

In [11]:
records1 = df.sample(1000).reset_index(drop=True)

In [12]:
create_type_1_dataset(records1, instructions, inputs, outputs, texts)

In [13]:
print(texts[134])

The following is an INSTRUCTION for a task.The paired INPUT below provides additional context for the task. Write a response to complete the task.

### INSTRUCTION:
Provide the details of the transaction made using the Card Number, Transaction Date, and the Date of Birth of the user given in the INPUT. The Response must include Merchant Name, Category, Transaction Id, and the Amount spent. If you do not have some of the information then output NA for those fields. If you have additional information output that as Others.

### INPUT:
Card Number: 2274911989136158
Transaction Date: 2020-01-19
Date of Birth: 1932-03-10

### RESPONSE:
Merchant: Thiel-Thiel
Category: entertainment
Transaction Id: af3df252a31f26008c907fbecb1e3345
Amount: $3.62
Others: 
</s>


In [14]:
records2 = df.sample(1000).reset_index(drop=True)

In [15]:
create_type_2_dataset(records2, instructions, inputs, outputs, texts)

In [16]:
print(texts[1453])

The following is an INSTRUCTION for a task. The paired INPUT below provides additional context for the task. Write a response to complete the task.

### INSTRUCTION:
Provide details of the credit card transaction made by a user. The First Name of the user, Transaction Date, and the Merchant Name are given in the INPUT context. Please provide the Last Name of the user, the Last 4 digits of the Credit Card number used, the Street Address and the Zipcode of the user. If you do not have some of the information then output NA for those fields. If you have additional information output that as Others.

### INPUT:
First Name: Candice
Transaction Date: 2019-09-02
Merchant Name: Beier-Hyatt

### RESPONSE:
Last Name: Brown
Last 4 digits of the Credit Card: 3927
Street Address: 9412 Harris Mews
Zipcode: 79539
Others: 
</s>


In [17]:
create_type_3_dataset(instructions, inputs, outputs, texts)

In [18]:
print(texts[2232])

The following is an INSTRUCTION for a task. The paired INPUT below provides additional context for the task. Write a response to complete the task.

### INSTRUCTION:
Find all Credit Card transactions made by the given user. The First Name and the Last Name of the user, Date of Birth, Street Address, City, State, Zipcode are given in the INPUT context. Please provide the Credit Card number, and all the known Transaction Dates, and the Amount charged for each Transaction. If you do not have some of the information then output NA for those fields.

### INPUT:
First Name: Veronica
Last Name: Burton
Date of Birth: 1991-08-21
Street Address: 08932 Bonnie Stravenue
City: Bonita Springs
State: FL
Zipcode: 34135


### RESPONSE:
Credit Card: 38544312728630
Transaction Date: 2019-01-02, Amount: $35.7
Transaction Date: 2019-01-03, Amount: $165.32
Transaction Date: 2019-01-05, Amount: $49.12
Transaction Date: 2019-01-06, Amount: $10.67
Transaction Date: 2019-01-06, Amount: $62.58
Transaction Date: 

In [19]:
traindf = pd.concat([records1, records2, cc_info_table], ignore_index=True).reset_index(drop=True)

In [20]:
traindf['Instructions'] = instructions
traindf['Input'] = inputs
traindf['Output'] = outputs
traindf['text'] = texts
traindf = traindf.sample(frac=1).reset_index(drop=True)
traindf

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,trans_num,unix_time,merch_lat,merch_long,is_fraud,transactions,Instructions,Input,Output,text
0,2019-05-18 14:13:46,4452366298769043,fraud_Auer LLC,personal_care,124.36,Linda,Davis,F,6602 Ortiz Pine Apt. 179,Blooming Grove,...,84d7575303dc4dc8c90458079824a9ea,1.337350e+09,32.141222,-97.544624,0.0,NaN,### INSTRUCTION:\nProvide the details of the t...,### INPUT:\nCard Number: 4452366298769043\nTra...,### RESPONSE:\nMerchant: Auer LLC\nCategory: p...,The following is an INSTRUCTION for a task.The...
1,NaN,570273151375,NaN,NaN,NaN,Peter,Edwards,NaN,96122 Steven Flat,Allentown,...,NaN,NaN,NaN,NaN,NaN,"[[2019-01-02 05:10:58, 70.45], [2019-01-08 06:...",### INSTRUCTION:\nFind all Credit Card transac...,### INPUT:\nFirst Name: Peter\nLast Name: Edwa...,### RESPONSE:\nCredit Card: 570273151375\nTran...,The following is an INSTRUCTION for a task. Th...
2,2020-04-16 16:48:15,3540210836308425,fraud_Gulgowski LLC,home,90.69,Steven,Yoder,M,94975 David Mews Apt. 316,Summerfield,...,76b58b66ee40248e823bc0ac4137ee21,1.366131e+09,34.463331,-102.727712,0.0,NaN,### INSTRUCTION:\nProvide the details of the t...,### INPUT:\nCard Number: 3540210836308425\nTra...,### RESPONSE:\nMerchant: Gulgowski LLC\nCatego...,The following is an INSTRUCTION for a task.The...
3,NaN,344709867813900,NaN,NaN,NaN,Joanna,Hudson,NaN,2924 Bobby Trafficway,Sebring,...,NaN,NaN,NaN,NaN,NaN,"[[2019-01-01 08:08:34, 185.1], [2019-01-01 11:...",### INSTRUCTION:\nFind all Credit Card transac...,### INPUT:\nFirst Name: Joanna\nLast Name: Hud...,### RESPONSE:\nCredit Card: 344709867813900\nT...,The following is an INSTRUCTION for a task. Th...
4,2019-07-06 07:20:49,2657850734909997,"fraud_Schaefer, Fay and Hilll",entertainment,2.80,Alan,Howard,M,04139 Johnson Prairie Suite 401,Detroit,...,cbed97ceaa12523a68ee501fd433b2d8,1.341559e+09,42.985158,-83.984601,0.0,NaN,### INSTRUCTION:\nProvide details of the credi...,### INPUT:\nFirst Name: Alan\nTransaction Date...,### RESPONSE:\nLast Name: Howard\nLast 4 digit...,The following is an INSTRUCTION for a task. Th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2978,2019-12-29 23:54:04,3576144910346950,"fraud_Mante, Luettgen and Hackett",health_fitness,9.03,Thomas,Payne,M,9455 Kathleen Brook Apt. 651,Huslia,...,c9d7bce4781e7e5df23e2f36581ed373,1.356825e+09,66.081246,-155.728250,0.0,NaN,### INSTRUCTION:\nProvide details of the credi...,### INPUT:\nFirst Name: Thomas\nTransaction Da...,### RESPONSE:\nLast Name: Payne\nLast 4 digits...,The following is an INSTRUCTION for a task. Th...
2979,2020-04-13 17:43:09,345389171551808,"fraud_Kuhic, Bins and Pfeffer",shopping_net,4.43,Justin,Fowler,M,5569 Phillips Neck Apt. 003,Coffeeville,...,d648ea6c7746a06c7534f266988a144d,1.365875e+09,34.163555,-90.331707,0.0,NaN,### INSTRUCTION:\nProvide details of the credi...,### INPUT:\nFirst Name: Justin\nTransaction Da...,### RESPONSE:\nLast Name: Fowler\nLast 4 digit...,The following is an INSTRUCTION for a task. Th...
2980,2020-02-15 22:14:06,3576431665303017,fraud_Klein Group,entertainment,211.15,Jessica,Ward,F,72269 Elizabeth Field Apt. 132,Phoenix,...,a1fa1d6266b38516d6d1111c6099763a,1.360966e+09,32.617465,-111.882683,0.0,NaN,### INSTRUCTION:\nProvide details of the credi...,### INPUT:\nFirst Name: Jessica\nTransaction D...,### RESPONSE:\nLast Name: Ward\nLast 4 digits ...,The following is an INSTRUCTION for a task. Th...
2981,2019-08-01 21:40:51,3533742182628021,fraud_Brown Inc,kids_pets,42.66,Robert,Haynes,M,857 Aaron Circles Suite 398,Johns Island,...,ad655edbdaabac9050f4a85764424aec,1.343857e+09,32.883640,-78.936302,0.0,NaN,### INSTRUCTION:\nProvide details of the credi...,### INPUT:\nFirst Name: Robert\nTransaction Da...,### RESPONSE:\nLast Name: Haynes\nLast 4 digit...,The following is an INSTRUCTION for a task. Th...


In [21]:
f"We have {len(df)} records, with unique credit cards={len(np.unique(df['cc_num']))}"

'We have 1296675 records, with unique credit cards=983'

In [22]:
traindf.to_csv('./credit_card_fraud_train_dataset.csv', index=False)

In [23]:
sanitydf = pd.read_csv('./credit_card_fraud_train_dataset.csv')

In [24]:
x=sanitydf.sample(10)
for msg in x['text']:
    print(msg)

The following is an INSTRUCTION for a task. The paired INPUT below provides additional context for the task. Write a response to complete the task.

### INSTRUCTION:
Provide details of the credit card transaction made by a user. The First Name of the user, Transaction Date, and the Merchant Name are given in the INPUT context. Please provide the Last Name of the user, the Last 4 digits of the Credit Card number used, the Street Address and the Zipcode of the user. If you do not have some of the information then output NA for those fields. If you have additional information output that as Others.

### INPUT:
First Name: Nathan
Transaction Date: 2020-03-26
Merchant Name: Beer-Jast

### RESPONSE:
Last Name: Martinez
Last 4 digits of the Credit Card: 7617
Street Address: 586 Thomas Cliffs
Zipcode: 54154
Others: 
</s>
The following is an INSTRUCTION for a task. The paired INPUT below provides additional context for the task. Write a response to complete the task.

### INSTRUCTION:
Provide d

In [25]:
inputs       = []
outputs      = []
instructions = []
texts        = []

In [26]:
records3 = df.sample(800).reset_index(drop=True)
records4 = records1.sample(200).reset_index(drop=True)
test_set_type_1 = pd.concat([records3, records4], ignore_index=True).reset_index(drop=True)
create_type_1_dataset(test_set_type_1, instructions, inputs, outputs, texts)

In [27]:
records5 = df.sample(800).reset_index(drop=True)
records6 = records2.sample(200).reset_index(drop=True)
test_set_type_2 = pd.concat([records5, records6], ignore_index=True).reset_index(drop=True)
create_type_2_dataset(test_set_type_2, instructions, inputs, outputs, texts)
create_type_3_dataset(instructions, inputs, outputs, texts)

In [28]:
testdf = pd.concat([test_set_type_1, test_set_type_2, cc_info_table], ignore_index=True).reset_index(drop=True)

In [29]:
testdf['Instructions'] = instructions
testdf['Input'] = inputs
testdf['Output'] = outputs
testdf['text'] = texts
testdf = testdf.sample(frac=1).reset_index(drop=True)

In [30]:
testdf.to_csv('./credit_card_fraud_test_dataset.csv', index=False)